In [1]:
# Import all the necessary packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
print(5)

5


In [4]:
all_comment = pd.read_csv('UScomments.csv', error_bad_lines= False)

FileNotFoundError: [Errno 2] No such file or directory: 'UScomments.csv'

In [ ]:
# This is the sample of comments from US YouTube for each video:
all_comment

In [ ]:
# Finding out missing values in the data:
all_comment.info()

In [ ]:
# Drop mmissing values:
all_comment.dropna(inplace = True)

In [ ]:
all_comment.isnull().sum()
# There is no null value in the data

### 1.Analyse the positive and negative comments with TextBlob and WordCloud:

In [ ]:
# Using TextBlob to create positive and negative comments in the data:

In [ ]:
from textblob import TextBlob

In [ ]:
# Example:
TextBlob('so good happy for you').sentiment.polarity

In [ ]:
# Create a column that contains the polarity of each comment:
polarity = []
for comment in all_comment['comment_text']:
    try:
        polarity.append(TextBlob(comment).sentiment.polarity)
    except:
        polarity.append(0)
        

In [ ]:
all_comment['polarity'] = polarity

In [ ]:
all_comment.head()

In [ ]:
cmt_positive = all_comment[all_comment['polarity'] == 1]
cmt_negative = all_comment[all_comment['polarity'] == -1]

In [ ]:
# This is the data where comments are positive (polarity = 1.0)
cmt_positive.head()

In [ ]:
# And this is the data of negative comments (polarity = -1.0)
cmt_negative.head()

In [ ]:
# Using WordCloud to show which words are used the most in positive and negative comments:

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
total_cmt_p = ''.join(cmt_positive['comment_text'])
total_cmt_n = ''.join(cmt_negative['comment_text'])

In [ ]:
wordcloud = WordCloud(stopwords= set(STOPWORDS)).generate(total_cmt_p)
plt.figure(figsize = (12,5))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('positive')

In [ ]:
wordcloud = WordCloud(stopwords= set(STOPWORDS)).generate(total_cmt_n)
plt.figure(figsize = (12,5))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('negative')

### 2.Performing Emoji analysis

In [ ]:
import emoji

In [ ]:
# Creat a list of all emoji used in the comments
emoji_list = []
for comment in all_comment['comment_text']:
    for char in comment:
        if char in emoji.UNICODE_EMOJI_ENGLISH:
            emoji_list.append(char)

In [ ]:
from collections import Counter
# This library help us to count the emoji values.

In [ ]:
common_emo = Counter(emoji_list).most_common(10)
# Count and extract top 10 common emoji used.

In [ ]:
common_emo

In [ ]:
emo = [common_emo[i][0] for i in range(10)]
emo
# Take out 10 emoji for x-axis values.

In [ ]:
freq = [common_emo[i][1] for i in range(10)]
freq
# Take out their count for y-axis values.

In [ ]:
# Now we have x-axis and y-axis so this is time for building a bar chart.
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
trace = go.Bar(x = emo, y = freq, width= 0.7)
iplot([trace])

#### Comment: As we can see, most of them are happy, like, encourage... emoji. We can see that viewers use more emoji when they love the video. The more they use emoji, the more they love your video. If people didnt use emoji in their comment, you can consider that if they don't like your video or not.

### (*) Collecting the YouTube video data

In [ ]:
# Import the additional video data first.

In [ ]:
# To do this we need OS library to take the all the file name we want:
import os

In [ ]:
path = r'./additional_data'

In [ ]:
file_name = os.listdir(path)
file_name
# Here are all the file name we want (This dataset only contain YouTube video data in some developed countries, not all the world)

In [ ]:
len(file_name)

In [ ]:
file_csv = [file_name[i] for i in range(0,20,2)]
file_csv

In [ ]:
df = pd.DataFrame()
for file in file_csv:
    current_df = pd.read_csv(path + '/' + file, error_bad_lines= False, encoding='iso-8859-1')
    # We need 'iso-8859-1' to encode the data  here to prevent errors.
    df = pd.concat([df, current_df])

In [ ]:
df.head()

### 3. Finding out which video category has the maximum likes

In [ ]:
# Import category file:
cat = pd.read_csv('category_file.txt', delimiter = ':')
# This step will help us creat a dictionary which contain category names and the ID of them.

In [ ]:
cat.head()

In [ ]:
cat.reset_index(inplace = True)

In [ ]:
cat.columns = ['Category_id', 'Category_name']

In [ ]:
cat = cat.set_index('Category_id')

In [ ]:
cat.head()

In [ ]:
cat_dict = cat.to_dict()

In [ ]:
cat_dict = cat_dict['Category_name']

In [ ]:
cat_dict
# Now we have the category dictionary we need.

In [ ]:
# From the dictionary we have just created, we add a column contain all the category names of videos in the df.
df['category_name'] = df['category_id'].map(cat_dict)

In [ ]:
df.head()
# We got that 'category_name' in the last of the frame.

In [ ]:
sns.set_theme(style = 'whitegrid') # Set white grid theme so we can see the chart more clearly.

In [ ]:
# Now it's time to create a box plot to see which category have most likes on YouTube:
plt.figure(figsize = (10,3))
sns.boxplot(data = df, x = 'category_name', y = 'likes')
plt.xticks(rotation = 'vertical')
plt.show()
plt.savefig('mostlikes.png')

In [ ]:
# We can finding out which categories have most views also:
plt.figure(figsize = (10,3))
sns.boxplot(data = df, x = 'category_name', y = 'views')
plt.xticks(rotation = 'vertical')
plt.savefig('mostviews.png')
plt.show()

#### Comment: From the box plot above, we can see that Music and Entertainment category have videos which have the maximum views and likes on YouTube. That mean most of people come to YouTube to enjoy the music videos and relax by entertainment ones. Trailers have less views and likes becaus they are not released frequently, and people may consider them as advertise videos.
#### If you want to have a better chance to have more views or likes on YouTube, try to make music contents such as create new songs, cover or remix famous songs with your own styles, realease your high quality MV... Or you can try entertainment content like doing some challenges, interesting things happen in your day, react to another videos...

### 4.Finding out whether the audience reacted with the videos or not

In [ ]:
df['likes_rate'] = df['likes']/df['views']*100
df['dislikes_rate'] = df['dislikes']/df['views']*100
df['comment_count_rate'] = df['comment_count']/df['views']*100

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (10,3))
sns.boxplot(data = df, x= 'category_name', y = 'likes_rate')
plt.xticks(rotation = 'vertical')
plt.show()

In [ ]:
plt.figure(figsize = (10,3))
sns.boxplot(data = df, x= 'category_name', y = 'comment_count_rate')
plt.xticks(rotation = 'vertical')
plt.show()

In [ ]:
plt.figure(figsize = (10,3))
sns.boxplot(data = df, x= 'category_name', y = 'dislikes_rate')
plt.xticks(rotation = 'vertical')
plt.show()

#### Comment: 3 box plots above show us how the viewers reacted to the videos. People used like button more frequently than dislike or leaving a comment. I can say that it is easier to like a video than dislike or comment on it. Because if people don't really like the video, even they hate it, they just can ignore it and watch another one which is more interesting. Some categories like 'News and Politic', 'People and Blogs', 'Nonprofit and Activism' which have high oppotunity to offense someone; or 'Entertainment', 'Gaming', 'Sience & Technology' which have rich contents for people to discuss, they have higher dislike rate or comment rate than others.
#### We can look at 'Gaming' and 'Sience & Technology' categories, they have high rate of like and comment but low rate of dislike. These categories are really adtractive and being loved by the people.So if you want your viewers have good reactions on your videos, try create videos about gaming, science or technology.

In [ ]:
# Analysing relationship between views and likes (dislikes, comment count):

In [ ]:
df[['views', 'likes', 'dislikes', 'comment_count']]

In [ ]:
sns.heatmap(df[['views', 'likes', 'dislikes', 'comment_count']].corr(), annot = True)

In [ ]:
## --> Views have a good correlation with Likes and have a poor correlation with Dislikes

In [ ]:
sns.regplot(data = df, x= 'views', y= 'likes', color= 'blue')
plt.show()

In [ ]:
sns.regplot(data = df, x= 'views', y= 'dislikes', color= 'red')
plt.show()

In [ ]:
sns.regplot(data = df, x= 'views', y= 'comment_count', color= 'green')
plt.show()

### 5. Which YouTube channels have the largest number of videos?

In [ ]:
# Create a data containing channel titles and the number of their videos
df_chan = df.copy()

In [ ]:
df_chan = df_chan.groupby('channel_title')['video_id'].count().sort_values(ascending = False)

In [ ]:
top10_chan = df_chan.head(10)

In [ ]:
top10_chan

In [ ]:
top10_chan = top10_chan.to_frame().reset_index().rename(columns = {'video_id':'total_video'})

In [ ]:
top10_chan

In [ ]:
# Using bar plot to analyse the data:
sns.barplot(data= top10_chan, x = 'channel_title', y = 'total_video')
plt.xticks(rotation = 'vertical')
plt.show()

#### Comment: It is obviously that the 'Late-night talk show' kinds such as 'The Late Show with Stephen Colbert', 'Late Night with Seth Meyers', 'TheEllenShow', 'Jimmy Kimmel Live'... have the lagrest number of videos. 'Late-night talk show' programs are very familiar with American since 1950s. And now they airs on YouTube mostly, so they have to release videos frequently to satisfy demand of audience. Most of these channels have a large investment and professional staff team. So as an individual YouTuber, you can pay attention to PewDiePie channel. PewDiePie is an YouTuber himself, he's making videos about gaming and his channel in the top biggest channel on YouTube.

In [ ]:
#Using plotly:

In [ ]:
import plotly.express as px

In [ ]:
px.bar(data_frame = top10_chan, x = 'channel_title', y = 'total_video')

### 7. Does Punctuations in title and tags have any relation with views, likes, dislikes comments?

In [ ]:
import string

In [ ]:
def count_punctuation(x):
    return len([i for i in x if i in string.punctuation])

In [ ]:
sample = df.head(10000)

In [ ]:
sample['punctuation'] = sample['title'].apply(count_punctuation)

In [ ]:
sample.head()

In [ ]:
plt.figure(figsize = (10,3))
sns.boxplot(data = sample, x = 'punctuation', y = 'views')
plt.show()

In [ ]:
plt.figure(figsize = (10,3))
sns.boxplot(data = sample, x = 'punctuation', y = 'likes')
plt.show()

In [ ]:
plt.figure(figsize = (10,3))
sns.boxplot(data = sample, x = 'punctuation', y = 'comment_count')
plt.show()

#### Comment: These chart show us that videos having 1-3 punctuations in their titles have more views, likes, and comments from the audience. It will make your title more adtractive and diferent from other videos. But using more than 8 punctuations in your video title can make the audience feel it's compicated and uncomfortable, so they could ignore your video.